In [1]:
import gradio as gr
from typing import Optional, List, Any
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, TextIteratorStreamer, AutoProcessor
# from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_core.outputs.generation import GenerationChunk
from torch import device
import time
from langchain.memory import ConversationBufferMemory



from threading import Thread
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 构建模型
class QianWenLLM(LLM):
    # 基于本地的QianWen7B-Chat模型自定义LLM类
    tokenizer: AutoTokenizer = None
    model: AutoModelForCausalLM = None
    processor: AutoProcessor = None
    
    def __init__(self, model_dir: str):
        # 从本地加载模型
        super().__init__()
        print('正从本地加载模型。。。。。')

        self.tokenizer = AutoTokenizer.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            trust_remote_code=True,
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            device_map='auto',
            trust_remote_code=True,
            torch_dtype=torch.bfloat16,
            # temperature=0.1
            )
        self.model = self.model.eval()
        self.model.generation_config = GenerationConfig.from_pretrained(
            model_dir,
            trust_remote_code=True
        )
        # 可指定不同的生成长度、top_p等相关超参
        self.processor = AutoProcessor.from_pretrained(model_dir)
        print('模型加载完成！')

    def _call(self, prompt: str,
              stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs: Any):
        # print('_call函数内查看prompt', prompt)
        response, history = self.model.chat(self.tokenizer, prompt, history=[])
        # response, history = self.model.generate(self.tokenizer, prompt, history=[])
        return response
        
    @property
    def _llm_type(self) -> str:
        return "QwenLM"

In [3]:
def init_model():
    # 初始化模型
    model_cache_path = r'autodl-tmp/Qwen/Qwen-7B-Chat'
    # model_cache_path = r'autodl-tmp/Qwen/Qwen2.5-7B-Instruct'
    llm = QianWenLLM(model_dir=model_cache_path)
    return llm

In [5]:
llm = init_model()
question = '请展示《上海联合产权交易所有限公司物权交易操作指引》的具体内容'
template = f"""
            回答输入的问题，如果你不知道答案，就说你不知道，不要试图编造答案。
            输入的问题：{question}
            有用的回答：
"""
llm.invoke(template.format(question))

正从本地加载模型。。。。。


The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

The repository for autodl-tmp/Qwen/Qwen-7B-Chat contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/autodl-tmp/Qwen/Qwen-7B-Chat.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository for autodl-tmp/Qwen/Qwen-7B-Chat contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/autodl-tmp/Qwen/Qwen-7B-Chat.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


模型加载完成！


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'非常抱歉，我无法提供您需要的内容。我是一个人工智能语言模型，我的知识库主要集中在公共领域的常见问题和一般性话题上。如果您有关于其他主题的问题，请随时告诉我，axe'

In [ ]:
sentence = [
            '你好',
            '你是谁？',
            '上海联合产权交易所有限公司物权交易操作指引(沪联产交〔2020〕26号)',
            '周杰伦是谁？',
            '产权交易争议调解',
            '2023年7月，中国航天科技集团公司五院研制的长征十二号运载火箭在酒泉卫星发射中心成功发射，将一颗北斗导航卫星送入预定轨道。',
            '周杰伦的歌曲《稻香》是谁写的？',
            '项目名称为ABC，中标金额为500万元，供应商为XYZ公司。']

In [ ]:
template = f"""
        你是一名实体提取和意图识别分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。
        
        任务：
        1-判断输入文本是否存在实体。
        2-抽取输入文本所有实体。
        3-根据实体与给出的意图标签进行判定该输入文本的意图。
        
        意图标签：政策知识，日常知识，招中标知识
        
        工作流程：
        1.-先判断是否存在实体，不存在实体则直接根据不存在实体输出格式输出，存在实体则继续以下工作流程，并通过存在实体输出格式输出。
        2-实体提取：请从输入的文本中提取出所有实体。
        3-意图分类：请根据第2点提取的实体以及意图标签，进行意图识别并分类输入文本。
        
        输入文本：{question}
        不存在实体输出格式：
            实体提取:[]
            意图分类:日常知识
        存在实体输出格式：
            实体提取：[实体1, 实体2, ...]
            意图分类：意图标签

        有用的回答：
       """